In [1]:
import pandas as pd

Inspeccionamos los datos

In [3]:
df_sg = pd.read_json(r'C:\Users\argui\OneDrive\Escritorio\ProyectoML_OPS\data\steam_games.json.gz',compression='gzip',lines=True)
df_sg.head(5) 

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


In [4]:
conteo_nulos = df_sg.isnull().sum()
conteo_nulos
#Al analizar los nulos, nos percatamos que tienen demasiados nulos, procederemos a limpiar los datos

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

In [5]:
df_sg = df_sg.dropna() #no especifico columna con el fin de optimizar las columnas también
df_sg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22530 entries, 88310 to 120443
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     22530 non-null  object 
 1   genres        22530 non-null  object 
 2   app_name      22530 non-null  object 
 3   title         22530 non-null  object 
 4   url           22530 non-null  object 
 5   release_date  22530 non-null  object 
 6   tags          22530 non-null  object 
 7   reviews_url   22530 non-null  object 
 8   specs         22530 non-null  object 
 9   price         22530 non-null  object 
 10  early_access  22530 non-null  float64
 11  id            22530 non-null  float64
 12  developer     22530 non-null  object 
dtypes: float64(2), object(11)
memory usage: 2.4+ MB


Eliminamos columnas que no nos sean útiles y renombramos las columnnas cuando sea necesario

In [6]:
df_sg = df_sg.drop(columns=['publisher','app_name','url','reviews_url','specs'])
df_sg = df_sg.rename(columns={'id':'id_game'})
df_sg.head(3)

,genres,title,release_date,tags,price,early_access,id_game,developer
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.99,0.0,761140.0,Kotoshiro
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",Free To Play,0.0,643980.0,Secret Level SRL
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",Free to Play,0.0,670290.0,Poolians.com


In [7]:
import re

#Creamos una función para extraer el año de la columba 'release_date'
def year(cadena:str) -> str|None:
    """
    La función year toma la cadena de texto y devuelve un año en formato entero
    o None si la cadena está vacía.
    """
    patron = r'(\d{4})'
    match = re.search(patron, str(cadena))
    if match:
        return match.group(1)
    else:
        return None
df_sg['release_date'] = df_sg['release_date'].apply(year)
df_sg['release_date'].head(3)

88310    2018
88311    2018
88312    2017
Name: release_date, dtype: object

Convertimos los datos tipo string a 0 en la columna price

In [8]:
#Creamos una función que transformelos los datos de la columa Price a float

def tofloat(precio:str) -> float:
    try:
        return float(precio)
    except (ValueError, TypeError):
        return 0.0
    
#Aplicamos

df_sg['price'] = df_sg['price'].apply(tofloat)
df_sg['price'].info()

<class 'pandas.core.series.Series'>
Index: 22530 entries, 88310 to 120443
Series name: price
Non-Null Count  Dtype  
--------------  -----  
22530 non-null  float64
dtypes: float64(1)
memory usage: 352.0 KB


Creamos una fila por cada dato que desanidamos en la columna 'genre'

In [9]:
df_sg = df_sg.explode('genres')
df_sg.head(5)

,genres,title,release_date,tags,price,early_access,id_game,developer
88310,Action,Lost Summoner Kitty,2018,"[Strategy, Action, Indie, Casual, Simulation]",4.99,0.0,761140.0,Kotoshiro
88310,Casual,Lost Summoner Kitty,2018,"[Strategy, Action, Indie, Casual, Simulation]",4.99,0.0,761140.0,Kotoshiro
88310,Indie,Lost Summoner Kitty,2018,"[Strategy, Action, Indie, Casual, Simulation]",4.99,0.0,761140.0,Kotoshiro
88310,Simulation,Lost Summoner Kitty,2018,"[Strategy, Action, Indie, Casual, Simulation]",4.99,0.0,761140.0,Kotoshiro
88310,Strategy,Lost Summoner Kitty,2018,"[Strategy, Action, Indie, Casual, Simulation]",4.99,0.0,761140.0,Kotoshiro


Por último exportamos a parquet para optimizar la velocidad de las consultas

In [10]:
df_sg.to_parquet(r'C:\Users\argui\OneDrive\Escritorio\ProyectoML_OPS\data\steam_games.parquet',compression='snappy',index=False)